In [52]:
import pandas as pd
import sqlite3


In [53]:
conn = sqlite3.connect('moderation.db')

df = pd.read_sql("select * from verdicts", conn)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16048 entries, 0 to 16047
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      16048 non-null  int64 
 1   banner_id  16048 non-null  int64 
 2   uid        16048 non-null  int64 
 3   verdict    16048 non-null  object
dtypes: int64(3), object(1)
memory usage: 501.6+ KB


In [54]:
df['verdict_int'] = (df['verdict'] == 'Yes').apply(int)

df

,index,banner_id,uid,verdict,verdict_int
0,0,57421551,51567,No,0
1,1,38788087,51567,No,0
2,2,27309982,51567,No,0
3,3,29100381,51567,No,0
4,4,72302555,51567,Yes,1
...,...,...,...,...,...
16043,16043,32418988,749066,Yes,1
16044,16044,32418980,749066,Yes,1
16045,16045,32418998,749066,Yes,1
16046,16046,32418982,749066,Yes,1


In [55]:
best_people = df.groupby('uid')



best_people = best_people.agg({'verdict':'count', 'verdict_int':'sum'})
#display(best_people)

best_people['success_per_cent'] = best_people['verdict_int'] / best_people['verdict']

best_people.sort_values('success_per_cent', inplace=True, ascending=False)

best_people.reset_index(inplace=True)

display(best_people)

#best_uid = best_people['uid']

display(df[df['uid'] == 373408])


,uid,verdict,verdict_int,success_per_cent
0,537791,98,97,0.989796
1,587328,64,63,0.984375
2,675837,53,52,0.981132
3,186507,50,49,0.980000
4,373408,50,49,0.980000
...,...,...,...,...
451,116939,49,2,0.040816
452,120370,30,1,0.033333
453,377512,49,1,0.020408
454,306833,50,1,0.020000


,index,banner_id,uid,verdict,verdict_int
3087,3087,48857118,373408,Yes,1
15448,15448,14867515,373408,No,0
15449,15449,14867205,373408,Yes,1
15450,15450,47038880,373408,Yes,1
15451,15451,29751859,373408,Yes,1
15452,15452,14867194,373408,Yes,1
15453,15453,14867522,373408,Yes,1
15454,15454,14866539,373408,Yes,1
15455,15455,14867014,373408,Yes,1
15456,15456,77180068,373408,Yes,1


In [71]:
def find_all_rejected(df: pd.DataFrame, sender_id: int):
    return df[(df['verdict'] == 'No') & (df['uid'] == sender_id)]['banner_id']


display(find_all_rejected(df, 373408))


prob_normal = pd.Series()

for index, row in best_people.iterrows():
    prob_normal = pd.concat([prob_normal, find_all_rejected(df, int(row['uid']))])
    #print('now row is', index, "row['uid'] is", int(row['uid']))
    #display(prob_normal)

    if len(prob_normal) > 100:
        break


display(pd.DataFrame(prob_normal[:100], columns=['banner_id']))
display(14867515 in prob_normal.values)




15448    14867515
Name: banner_id, dtype: int64

/tmp/ipykernel_8042/3583767058.py:11: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  prob_normal = pd.concat([prob_normal, find_all_rejected(df, int(row['uid']))])


,banner_id
15133,32363093
15557,32369662
12207,62639695
11466,69609075
15448,14867515
...,...
74,32344224
13573,43486552
8716,44606170
8717,44606154


True

In [57]:
import pandas as pd



def get_sample(df: pd.DataFrame) -> pd.DataFrame:
    df['verdict_int'] = (df['verdict'] == 'Yes').apply(int)


    best_people = df.groupby('uid')
    best_people = best_people.agg({'verdict':'count', 'verdict_int':'sum'})

    best_people['success_per_cent'] = best_people['verdict_int'] / best_people['verdict']
    best_people.sort_values('success_per_cent', inplace=True, ascending=False)
    best_people.reset_index(inplace=True)



    def find_all_rejected(df: pd.DataFrame, sender_id: int):
        return df[(df['verdict'] == 'No') & (df['uid'] == sender_id)]['banner_id']

    prob_normal = pd.Series()
    for index, row in best_people.iterrows():
        prob_normal = pd.concat([prob_normal, find_all_rejected(df, int(row['uid']))])

        if len(prob_normal) > 100:
            break


    return pd.DataFrame(prob_normal[:100], columns=['banner_id'])




In [72]:
res = get_sample(df)
display(res)

14867515 in res['banner_id'].values

/tmp/ipykernel_8042/278588316.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  prob_normal = pd.concat([prob_normal, find_all_rejected(df, int(row['uid']))])


,banner_id
15133,32363093
15557,32369662
12207,62639695
11466,69609075
15448,14867515
...,...
74,32344224
13573,43486552
8716,44606170
8717,44606154


True